# Procesamiento de datos del viejo y nuevo Congreso

## Preparación

Abrir las tablas de datos históricos del Directorio Legislativo (2000-2018).

Crear columna con el nombre completo que concatene el campo nombre con el campo apellido y en base a el nombre eliminar duplicados.

In [87]:
import pandas as pd
import unidecode
import warnings
warnings.filterwarnings('ignore')

diputados = pd.read_csv('data/diputados_2000_2018.csv')
diputados['nombre'] = diputados['diputado_nombre'] + " " + diputados['diputado_apellido']
diputados['nombre'] = diputados['nombre'].apply(lambda x: unidecode.unidecode(x.decode('utf-8')).lower())
diputados_todos = diputados.sort_values('diputado_mandatoFinal', ascending=False).groupby('nombre').first().reset_index()

senadores = pd.read_csv('data/senadores_2000_2018.csv')
senadores['nombre'] = senadores['datospersonales_nombre'] + " " + senadores['datospersonales_apellido']
senadores['nombre'] = senadores['nombre'].apply(lambda x: unidecode.unidecode(x.decode('utf-8')).lower())
senadores_todos = senadores.sort_values('datospersonales_mandatoFinal', ascending=False).groupby('nombre').first().reset_index()

Abrir listado de precandidatos con corrección de nombres.

Dado que en varios casos se omite el uso del segundo nombre, se hizo una corrección usando distancias de Levenshtein e indicando manualmente las coincidencias en [este archivo](https://docs.google.com/spreadsheets/d/1MYXn653RUNsCIba9UrASMbzlbJBx0YZYb0b8mPf1Tpc/edit#gid=0).

In [82]:
listas = pd.read_csv('data/nombres_legisladores_match.csv')

listas['nombre_final'] = listas.apply(lambda x: x['result_l'] if x['misma_persona'] == 'Si' else x['nombre'], axis=1)

Separar los legisladores vigentes de los del pasado a partir del año de su mandato final (2019 y 2021 para los diputados y 2019, 2021 y 2023 para senadores).

In [83]:
diputados_vigente = diputados_todos[diputados_todos['diputado_mandatoFinal'].isin([2019, 2021])]
diputados_pasado = diputados_todos[~diputados_todos['diputado_mandatoFinal'].isin([2019, 2021])]

senadores_vigente = senadores_todos[senadores_todos['datospersonales_mandatoFinal'].isin([2019, 2021, 2023])]
senadores_pasado = senadores_todos[~senadores_todos['datospersonales_mandatoFinal'].isin([2019, 2021, 2023])]

## Grupos

Marcar en los legisladores vigentes si renuevan fijandose si están presentes en el listado de precandidatos y marcar en los legisladores del pasado si vuelven fijandose lo mismo.

A partir de esto se deduce que los que no renuevan y terminan mandato en 2019, se van.

In [90]:
diputados_vigente['renueva'] = diputados_vigente['nombre'].isin(listas['nombre_final'])
senadores_vigente['renueva'] = senadores_vigente['nombre'].isin(listas['nombre_final'])

diputados_vigente['se_va'] = (~diputados_vigente['renueva']) & (diputados_vigente['diputado_mandatoFinal'] == 2019)
senadores_vigente['se_va'] = (~senadores_vigente['renueva']) & (senadores_vigente['datospersonales_mandatoFinal'] == 2019)

diputados_pasado['vuelve'] = diputados_pasado['nombre'].isin(listas['nombre_final'])
senadores_pasado['vuelve'] = senadores_pasado['nombre'].isin(listas['nombre_final'])

print('Renuevan {} diputados y {} senadores'.format(len(diputados_vigente[diputados_vigente['renueva']]), len(senadores_vigente[senadores_vigente['renueva']])))

print('Se van {} diputados y {} senadores'.format(len(diputados_vigente[diputados_vigente['se_va']]), len(senadores_vigente[senadores_vigente['se_va']])))

print('Vuelven {} diputados y {} senadores'.format(len(diputados_pasado[diputados_pasado['vuelve']]), len(senadores_pasado[senadores_pasado['vuelve']])))

Renuevan 48 diputados y 8 senadores
Se van 101 diputados y 21 senadores
Vuelven 32 diputados y 10 senadores


## Dimensiones

Se evaluarán los grupos según las siguientes dimensiones:
* Edad
* Género
* Nivel de estudios
* Actividad privada previa
* Actividad pública previa